# Dipole Example (under development)

Simple dipole example


In [ ]:
from impact import Impact

import numpy as np
import os

import matplotlib.pyplot as plt

%config InlineBackend.figure_format='retina'

In [ ]:
# locate the drift template
ifile = "../templates/dipole/ImpactT.in"
os.path.exists(ifile)

In [ ]:
I = Impact(ifile)
I.header["Bcurr"] = 0
# I.total_charge=1e-12
# I.stop = 2.5
I.numprocs = 8
I.run()

c_light = 299792458.0  # m/s
(
    I.particles["final_particles"]["mean_t"] * c_light,
    I.particles["final_particles"]["mean_z"],
)

In [ ]:
I.stat("mean_z")

In [ ]:
I.plot("mean_x")

In [ ]:
!cat {I.path}/ImpactT.in

In [ ]:
# calculate gamma*beta
mec2 = 0.51099895000e6  # eV
Etot = 1e9  # eV
gamma = Etot / mec2
GB = np.sqrt(gamma**2 - 1)
GB

In [ ]:
from impact.elements.dipole import new_dipole_fieldmap_data, dipole_fieldmap_lines

# THETA = 30*np.pi/180
MYL = 2.1
# MYG = THETA/MYL
GAMMA = 1e9 / mec2
MYG = 1 / 10
# GAMMA = 1e9/mec2
THETA = MYG * MYL


GB = np.sqrt(GAMMA**2 - 1)

DMAP = new_dipole_fieldmap_data(
    L=MYL, g=MYG, gamma_ref=GAMMA, e1=0, e2=0, half_gap=0.01, fint=0.5, csr_on=True
)

# make fieldmap
print(dipole_fieldmap_lines(DMAP, "../templates/dipole/rfdata777"))

In [ ]:
DMAP

# Use Impact's built-in Gaussian particle generator

In [ ]:
I = Impact(ifile)

c_light = 299792458.0

I.header["Np"] = 1000000
I.header["Nx"] = 32
I.header["Ny"] = 32
I.header["Nz"] = 256
I.header["Dt"] = 0.01 / c_light
# I.header['Ntstep'] = 200
# I.input['fieldmaps']['rfdata777']['data'][0] = 1 # Turn on/off CSR
# I.input['fieldmaps']['rfdata777']['data'][-1] = 1 # exit_s
I.ele["B1"]["L"] = MYL

# I.header['zmu2'] = GB
I.numprocs = 8
I.stop = 3.1
I.total_charge = 1e-12
I.verbose = True
I.write_input()

print(I)

In [ ]:
!cat {I.path}/ImpactT.in

In [ ]:
%%time
I.run()

In [ ]:
I.particles

In [ ]:
# afile1 = I.archive()

In [ ]:
PF = I.particles["final_particles"]
PF["mean_z"]

In [ ]:
P26 = PF.copy()

In [ ]:
PF.plot("delta_z", "delta_p", bins=100, figsize=(12, 8))

In [ ]:
P2 = PF.copy()

In [ ]:
P1["mean_z"], P2["mean_z"]

In [ ]:
k1 = "min_p"
P1[k1], P4[k1], P3[k1]

In [ ]:
k1 = "delta_z"
k2 = "p"
plt.scatter(P1[k1], P1[k2])
plt.scatter(P2[k1], P2[k2])

In [ ]:
I.particles["initial_particles"].plot("y", "py")

# Single particle tracking

In [ ]:
from impact.elements import dipole

In [ ]:
dipole.parse_dipole_fieldmap_data

In [ ]:
I.ele["B1"]

In [ ]:
DATA = I.input["fieldmaps"][I.ele["B1"]["filename"]]["data"]
DMAP = dipole.parse_dipole_fieldmap_data(DATA)
DMAP

In [ ]:
I.track1(pz0=1e9)
I.path

In [ ]:
I.header["Flagdist"]

In [ ]:
dipole.plot_dipole_fieldmap(DMAP, ref_data=I.output["dipole_stats"], g=1 / 10, L=3.0)

In [ ]:
I.input["fieldmaps"]["rfdata777"]["data"][0] = 1

In [ ]:
I.verbose = False
I.run()

# CSR analysis

In [ ]:
%%time
I1 = I.copy()
I1.configure()
I1.stop = 2.5
I1.verbose = False
I1.run()

I1.particles["final_particles"]["mean_t"] * c_light

In [ ]:
!cat {I1.path}/ImpactT.in

In [ ]:
I2 = I.copy()
I2.configure()
I2.stop = 2.6
I2.verbose = False
I2.run()

In [ ]:
, I2.particles['final_particles']['mean_t']*c_light

In [ ]:
from scipy.constants import physical_constants

mec2 = physical_constants["electron mass energy equivalent in MeV"][0] * 1e6

c_light = 299792458.0  # m/s

r_electron = physical_constants["classical electron radius"][0]


def W0_calc(rho=10, sigma_z=0.0003, total_charge=1e-12):
    N = total_charge / 1.6e-19

    return N * r_electron * mec2 / rho ** (2 / 3.0) / sigma_z ** (4 / 3.0)


W0 = W0_calc()

W0

In [ ]:
# I1 = Impact.from_archive('impact_6bcb411d0efbbf51d6df8096bdc4dd0d.h5') # 2.5 m

# I2 = Impact.from_archive('impact_8abe9fea7069035129b86b72021a3957.h5') # 2.6 m

P1 = I1.particles["final_particles"]
z1 = P1["mean_z"]
P1.z -= z1

P2 = I2.particles["final_particles"]
z2 = P2["mean_z"]
P2.z -= z2

z1, z2

In [ ]:
from pmd_beamphysics.statistics import slice_statistics

?slice_statistics

In [ ]:
dat1 = slice_statistics(P1, keys=["mean_z", "mean_energy"])
dat2 = slice_statistics(P2, keys=["mean_z", "mean_energy"])

In [ ]:
plt.plot(dat1["mean_z"], dat1["mean_energy"])

In [ ]:
Z = np.linspace(-6e-4, 6e-4, 100)
E1 = np.interp(Z, dat1["mean_z"], dat1["mean_energy"])
E2 = np.interp(Z, dat2["mean_z"], dat2["mean_energy"])

In [ ]:
dz = z2 - z1
dz

In [ ]:
plt.plot(Z, E1)
plt.plot(Z, E2)

In [ ]:
plt.xlabel("$z/\sigma_z$")
plt.ylabel("$dE/dz/W_0$")
plt.plot(Z / 0.0003, (E2 - E1) / dz / W0)

In [ ]:
# Mathematica computed
SSdat = np.loadtxt("../../devel/steadystate_csr_wake.dat").T
SSz = SSdat[0]
SSw = SSdat[1]
plt.plot(SSz, SSw)

In [ ]:
plt.xlabel("$z/\sigma_z$")
plt.ylabel("$dE/dz/W_0$")
plt.scatter(Z / 0.0003, (E2 - E1) / dz / W0, label="Impact-T", color="red")
plt.plot(SSz, SSw, label="1-D model", color="black")
plt.legend()

In [ ]:
0.02 ** (2 / 3) * 2.5

In [ ]:
(0.0006) ** (-2 / 3) * (100.0) ** (-1 / 3) * 0.0254 / 2

In [ ]:
W0_calc(rho=10, sigma_z=0.05, total_charge=1e-9) / 2.5e6